# Proyecto

Autor: Fabian Morera Gutierrez

Profesor: Dr. Juan Manuel Esquivel Rodriguez

Curso: Big Data

# Entrenamiento de dos modelos de clasificación binaria para una variable de crítica especializada para un set de VideoJuegos.

El presente programa tiene como finalidad la creación de dos modelos de clasificaciónbinaria, de principio a fin, utilizando el framework Apache Spark. Para tal fin, se utilizará un set de datos con información de ventas y crítica de videojuegos a nivel mundial.

Se asume que dicho conjunto de datos fue previamente depurado, limpiado y preparado para su futura utilización como set de entrenamiento y testeo de un modelo.

## Dependencias
Importamos dependencias requeridas a lo largo del programa aquí.

In [ ]:
# Numpy y Matplotlib.
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

# Others.
import findspark
import tempfile
import random

# Pyspark main.
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Pyspark machine learning.
from pyspark.ml.classification import LogisticRegression
from pyspark.mllib.regression import LabeledPoint
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import CrossValidatorModel, CrossValidator, ParamGridBuilder
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler, HashingTF, Tokenizer, StandardScaler
from pyspark.ml import Pipeline

from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

# Seaborn.
import seaborn as sns

print("Dependencies loaded.")

# Inspección de datos
Previo a entrenar el modelo es común que se realice algún tipo de descripción de los datos, para tener una idea del tipo de problema con el que nos enfrentamos. A continuación, algunas observaciones interesantes.

In [ ]:
findspark.init('/usr/lib/python3.7/site-packages/pyspark')

# Cargar el conjunto de datos completo. Este paso no realiza ningún ajuste; simplemente lectura
spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.driver.extraClassPath", "postgresql-42.2.14.jar") \
    .config("spark.executor.extraClassPath", "postgresql-42.2.14.jar") \
    .getOrCreate()

# Cargar el conjunto de datos desde la base de datos (Tabla vg_critic_sales).
videogames_df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://host.docker.internal:5433/postgres") \
    .option("user", "postgres") \
    .option("password", "testPassword") \
    .option("dbtable", "vg_critic_sales") \
    .load()

print("Data loaded!")
videogames_df.show()

## Información Descriptiva de los Datos.
Se muestran histogramas y resúmenes descriptivos de algunas de las variables disponibles.

In [ ]:
# Información descriptiva de alguno valores relevantes del dataframe.
videogames_df.describe([
    'metascore', 
    'metacritic_userscore', 
    'total_shipped']).show()